##Instalação obrigatória

In [ ]:
#pip install --upgrade pip
#pip install apache_beam[interactive]
#pip install apache_beam[gcp]

##Ler e Exibir a leitura

AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED'

In [ ]:
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Imprimir o resultado'>> beam.Map(print)
)

p1.run()

#Criação de arquivo

In [ ]:
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=0)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    #|'Imprimir o resultado'>> beam.Map(print)
    |'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

p1.run()

##Filtros


In [ ]:
#Filtrar o aero origem SFO
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro aero'>> beam.Filter(lambda record: record[1] == 'SFO')
    |'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

p1.run()

In [ ]:
#Filtrar saida atrasada
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[5])>0)
    |'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

p1.run()

In [ ]:
#Filtrar Voo atrasado
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[16])>0)
    |'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

p1.run()

##Agrupamento

In [ ]:
#Agrupamento de minutos de atraso por aero
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[16])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[1],float(record[16])))
    |'construir tabela por minutos somados'>> beam.CombinePerKey(sum)
    |'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

p1.run()

In [ ]:
#Agrupamento ocorrencias de atraso por aero
import apache_beam as beam

p1 = beam.Pipeline()

voos = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[16])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[1],float(record[16])))
    |'construir tabela por minutos somados'>> beam.combiners.Count.PerKey()
    |'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

p1.run()

##CoGroupbykey

In [ ]:
#Criar nova tabela com dados agregados de duas pipes
import apache_beam as beam

p1 = beam.Pipeline()

qte_tempo = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[16])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[1],float(record[16])))
    |'construir tabela por minutos somados'>> beam.CombinePerKey(sum)
    #|'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

qte_ocor = (
    p1
    |'1Extrair do CSV'>> beam.io.ReadFromText('/content/voos.csv',skip_header_lines=1)
    |'1Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'1Filtro atraso'>> beam.Filter(lambda record: float(record[16])>0)
    |'1Agregar as colunas'>> beam.Map(lambda record: (record[1],float(record[16])))
    |'1construir tabela por minutos somados'>> beam.combiners.Count.PerKey()
    #|'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

tabela = (
    {'Num_de_ocorrencias':qte_ocor,'Quant_minutos':qte_tempo}
    |'Agrupar as pernas'>> beam.CoGroupByKey()
    #|'Resultado  final'>> beam.Map(print)
    |'Load final'>> beam.io.WriteToText('tabelafim.txt')
)
p1.run()

##Criação de model para GCP

In [ ]:
import apache_beam as beam
import os
from apache_beam.options.pipeline_options import PipelineOptions

pipeline_options = {
    'project':'bc26-aulas-368614',
    'runner':'DataflowRunner',
    'region':'southamerica-east1',
    'staging_location':'gs://bc26-datasets/staging/',
    'temp_location':'gs://bc26-datasets/temp/',
    'template_location':'gs://bc26-datasets/models/modelo_batch'
}

serviceAccount = '/content/bc26-aulas-368614-7b5657fd555a.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)

p1 = beam.Pipeline(options=pipeline_options)

qte_tempo = (
    p1
    |'Extrair do CSV'>> beam.io.ReadFromText('gs://bc26-datasets/brutos/voos.csv',skip_header_lines=1)
    |'Separador de dados'>> beam.Map(lambda record: record.split(','))
    |'Filtro atraso'>> beam.Filter(lambda record: float(record[16])>0)
    |'Agregar as colunas'>> beam.Map(lambda record: (record[1],float(record[16])))
    |'construir tabela por minutos somados'>> beam.CombinePerKey(sum)
    #|'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

qte_ocor = (
    p1
    |'Abrir do CSV'>> beam.io.ReadFromText('gs://bc26-datasets/brutos/voos.csv',skip_header_lines=1)
    |'Separar por virgula'>> beam.Map(lambda record: record.split(','))
    |'Filtro aero atraso'>> beam.Filter(lambda record: float(record[16])>0)
    |'Juntar as colunas'>> beam.Map(lambda record: (record[1],float(record[16])))
    |'construir tabela por ocorrencia'>> beam.combiners.Count.PerKey()
    #|'Imprimir o resultado'>> beam.Map(print)
    #|'Criar arquivo'>> beam.io.WriteToText('saidaerro.txt')
)

tabela = (
    {'Num_de_ocorrencias':qte_ocor,'Quant_minutos':qte_tempo}
    |'Agrupar as pernas'>> beam.CoGroupByKey()
    #|'Resultado  final'>> beam.Map(print)
    |'Load final'>> beam.io.WriteToText('gs://bc26-datasets/tratado/voosatr',file_name_suffix='.csv')
)
p1.run()

<DataflowPipelineResult None at 0x7fcff346f580>